## Guess column names

Problem is to recognise column names. Lets start by cheating. Generate dataset and guess column names on that.

## Generate Fake data

Test faking code is working
faker is from https://github.com/joke2k/faker


In [2]:
from faker import Faker
fake = Faker()
Faker.seed(4321)#make a sseed for reproducability


fake.name()
# 'Lucy Cechtelar'

fake.address()
# '426 Jordy Lodge
#  Cartwrightshire, SC 88120-6700'

fake.text()
# 'Sint velit eveniet. Rerum atque repellat voluptatem quia rerum. Numquam excepturi
#  beatae sint laudantium consequatur. Magni occaecati itaque sint et sit tempore. Nesciunt
#  amet quidem. Iusto deleniti cum autem ad quia aperiam.
#  A consectetur quos aliquam. In iste aliquid et aut similique suscipit. Consequatur qui
#  quaerat iste minus hic expedita. Consequuntur error magni et laboriosam. Aut aspernatur
#  voluptatem sit aliquam. Dolores voluptatum est.
#  Aut molestias et maxime. Fugit autem facilis quos vero. Eius quibusdam possimus est.
#  Ea quaerat et quisquam. Deleniti sunt quam. Adipisci consequatur id in occaecati.
#  Et sint et. Ut ducimus quod nemo ab voluptatum.'

'Today project attorney help any other position scene. Half participant instead without medical around appear.\nDebate bar gas enough about ever. I force back all size speech.'

In [3]:
fake.name()

'Anna Patterson'

In [54]:
#imports
from random import randint
import csv
import notebook as nb
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [196]:
labeled = pd.DataFrame(columns=['name', 'address','text','boo','date','phone','ssn'])

for _ in range(100):
    name=fake.name()
    address=fake.address()
    text= fake.text()
    boo= randint(0, 1)
    job=fake.job()
    date = fake.date_time()  
    phone=fake.phone_number()
    ssn=fake.ssn()
    company=fake.company()
    #ip=fake.internet()
    labeled = labeled.append({'name' : name ,'address':address,'text':text,'boo':boo,'date':date,'phone':phone,'ssn':ssn} , ignore_index=True)
    


In [197]:
#check output looks reasonable
labeled.head()

,name,address,text,boo,date,phone,ssn
0,William Hendricks,"85529 Gregory Isle Suite 504\nNathanmouth, MT ...",Three speak full about around in development. ...,1,1983-12-11 17:39:57,(441)725-6346,207-24-6696
1,Erika Moore,"885 Nathaniel Drives\nEast Tanyamouth, MN 79262",Much admit similar already personal shoulder l...,0,2001-09-25 23:26:43,384.840.0153x37173,492-56-8778
2,Patricia Johnson MD,USNV Wilson\nFPO AE 04817,Join reflect suffer Democrat again capital. Va...,0,2005-08-03 09:23:48,001-278-086-2522x71616,749-47-5652
3,Brenda Davis,"2653 Christopher Spring\nNew Jessicaberg, VT 2...",Perform day eat someone off fall. Everything s...,1,1978-02-16 11:26:32,265-964-5816x333,390-35-4995
4,Stacey Nelson,"4639 Harrington Locks Suite 909\nKanehaven, MT...",Very support bit quickly section less state to...,0,2016-10-03 21:32:54,(450)687-7726x10575,638-45-7084


## Get spacy loaded 

In [20]:
import spacy
nlp=spacy.load('en')

#nlp = spacy.load("en_core_web_sm")

In [21]:
#check spacy is working
nlp=spacy.load('en')

#nlp = spacy.load("en_core_web_sm")
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Apple 0 5 ORG
U.K. 27 31 GPE
$1 billion 44 54 MONEY


## Get column types

In [216]:
g = labeled.columns.to_series().groupby(labeled.dtypes).groups

g

{dtype('<M8[ns]'): Index(['date'], dtype='object'),
 dtype('O'): Index(['name', 'address', 'text', 'boo', 'phone', 'ssn'], dtype='object')}

In [219]:
#del labeled['date']
labeled.columns

Index(['name', 'address', 'text', 'boo', 'phone', 'ssn'], dtype='object')

In [220]:
#Find only numeric columns
labeled.apply(lambda s: pd.to_numeric(s, errors='coerce').notnull().all())

name       False
address    False
text       False
boo         True
phone      False
ssn        False
dtype: bool

In [231]:
count=0
if labeled[labeled['ssn'].str.match('^(\d{3}-?\d{2}-?\d{4}|XXX-XX-XXXX)$')== True]:
    count+=1

print(count)    

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [227]:
#look for ssn
#labeled['ssn'].count(r'(^F.*)').sum()
#labeled.apply(lambda s: pd.to_numeric(s, errors='coerce').notnull().all()) == True
labeled[labeled['ssn'].str.match('^(\d{3}-?\d{2}-?\d{4}|XXX-XX-XXXX)$').count]
#

AttributeError: 'RangeIndex' object has no attribute 'levels'

## Now see Spacy detects columns correctly

In [198]:
#data is in labeled 
#how many rows in labeled
rows=len(labeled.index)
cols=len(labeled.columns)


In [199]:
labeled.iloc[2, 0]

'Patricia Johnson MD'

In [206]:

import re


     
    
def printme(j, guesses,finger):
    print("column number ",j)
    fingerprint=guesses.groupby('first').size()
    #print(fingerprint)
    #fingerprint=str(fingerprint)
    #print(fingerprint['first'].extract(r'.*PERSON.*'))
    #print("and person found")
    #print(fingerprint.astype(str).str.extract(r'.*PERSON(.*\d*\n)'))
    try:
        person=fingerprint['PERSON']
        number=fingerprint['CARDINAL']
        location=fingerprint['GPE']
        company=fingerprint['ORG']
        date=fingerprint['DATE']
        finger = finger.append({'column':j,'person' : person ,'number':number,'location':location,'company':company,'date':date,'phone':0,'ssn':0} , ignore_index=True)
   
        #print(person)
        #raise ErrorType("Error message")
    
    #Code that may raise an error
    except KeyError as e:#as e:IndexError
        print("key error")
    
    #print(fingerprint.astype(str).str.count(r'.*PERSON.*'))
    #print(fingerprint.count(r'.*PERSON.*'))    
    #x = re.search(".*PERSON.*", fingerprint)
    #print(x)
    #print(guesses['first'].value_counts())
    return finger

In [169]:
fingerprint=guesses.groupby('first').size()
fingerprint

first
PERSON    96
dtype: int64

In [175]:
fingerprint.index

Index(['PERSON'], dtype='object', name='first')

In [170]:
print(type(fingerprint))

<class 'pandas.core.series.Series'>


In [174]:
#fingerprint[0]
fingerprint['PERSON']

96

In [207]:
#guess first column

finger = pd.DataFrame(columns=['column','person', 'number','location','company','date','phone','ssn'])


guesses= pd.DataFrame()

i = 0
j=0
while j< cols:
    while i < rows:
        #print(labeled.iloc[i, j])
        col1 = nlp(str(labeled.iloc[i, j]))#read in the row 
        i += 1
        for ent in col1.ents:#extract entities using spacy
            first=ent.label_
        #print(ent.label_)  
            new_row={'first':first}
            guesses = guesses.append(new_row, ignore_index=True)
    j += 1
    i = 0

    #print(j)
    printme(j,guesses,finger)

    guesses=pd.DataFrame()
    #for ent in col2.ents:#extract entities using spacy
        #print(ent.text, ent.start_char, ent.end_char, ent.label_)
        #second=ent.label_
        #print(ent.label_,col2)  
        #new_row={'first':first,'second':second}
        #guesses = guesses.append(new_row, ignore_index=True)
 





column number  1
key error
column number  2
column number  3
column number  4
key error
column number  5
key error
column number  6
key error
column number  7
key error


now extract from this size the values we want

In [ ]:
guesses['first'].value_counts(bins=4)


In [68]:
g = guesses.groupby('second')
size = g.size()
size[size > 20]

second
CARDINAL    101
DATE         41
GPE          43
ORG          46
PERSON       38
dtype: int64